In [2]:
import os
import gensim.models as models
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import random,numpy as np
import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize
from nltk.corpus import stopwords
# from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
import operator, random
import sys
import math
import pickle
from nltk.stem import *
from num2words import num2words
from wordcloud import STOPWORDS

def remove_header_footer(final_string):
	new_final_string=""
	flag=1
	tokens=final_string.split('\n\n')
	# Remove tokens[0] and tokens[-1]
	for token in tokens[1:-1]:
		flag+=1
		new_final_string+=str(token)+" "
	flag=0
	return new_final_string

def remove_html(data):
	return BeautifulSoup(data, "html.parser").get_text()

# def remove_btw_sqr(data):
#     fin = re.sub('\[[^]]*\]', '', data)
#     return fin

def fix_contractions(data):
    fin = contractions.fix(data)
    return fin

def words_tokenizer(data):
	words = nltk.word_tokenize(data)
	# tknzr = TweetTokenizer()	
	# tknzr.tokenize(data)
	return words

def remove_non_ascii(words):
	new_words = []
	flag = 0
	for i in range(len(words)):
		flag = 1
		new_word = unicodedata.normalize('NFKD',unicode(words[i]))
		new_word = new_word.encode('ascii','ignore')
		new_word = new_word.decode('utf-8','ignore')
		flag+=1
		new_words.append(new_word)
	return new_words
# def remove_non_ascii(words):
#     new_words = []
#     flag = 0
#     for i in range(len(words)):
#  		flag=1
#  		new_word = unicodedata.normalize('NFKD', words[i]).encode('ascii', 'ignore').decode('utf-8', 'ignore')
#  		flag+=1
#  		new_words.append(new_word)
#  	return new_words

def to_lowercase(words):
    new_words = []
    flag = 0
    for i in range(len(words)):
        new_word = words[i].lower()
        flag+=1
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    new_words = []
    flag = 0
    for i in range(len(words)):
    	flag+=1
        new_word = re.sub(r'([^\w\s])|_+', '', words[i])
        if new_word != '':
        	flag=0
        	new_words.append(new_word)
    return new_words

# def replace_numbers(words):
#     p = inflect.engine()
#     new_words = []
#     flag = 0
#     for i in range(len(words)):
#     	flag = 1
#         if words[i].isdigit():
#             new_word = p.number_to_words(words[i])
#             flag+=1
#             new_words.append(new_word)
#         else:
#         	flag = 0
#         	new_words.append(words[i])
#     return new_words

def replace_numbers(words):
	new_words = []
	for i in range(len(words)):
		if words[i].isdigit():
			temp_word = num2words(words[i])
			new_words.append(temp_word)
		else:
			new_words.append(words[i])
	return new_words

def remove_stopwords(words):
    new_words = []
    flag = 0
    for i in range(len(words)):
    	flag = 1
        if words[i] not in stopwords.words('english') and words[i] not in STOPWORDS:
        	flag+=1
        	new_words.append(words[i])
    return new_words

def remove_numbers(words):
    new_words = []
    flag=0
    for i in range(len(words)):
        if words[i].isdigit():
            flag+=1
        else:
            new_words.append(words[i])
    return new_words

def stemming(words):
	new_words = []
	stemmer = PorterStemmer()
	for i in range(len(words)):
		new_words.append(stemmer.stem(words[i]))
	return new_words

def preprocess_input_sentence(data):
	# data = remove_header_footer(data)
	data = remove_html(data)
	# data = remove_btw_sqr(data)
	data = fix_contractions(data)
	words = words_tokenizer(data)
	words = remove_non_ascii(words)
	words = to_lowercase(words)
	words = remove_punctuation(words)
	words = replace_numbers(words)
	words = stemming(words)
	words = remove_stopwords(words)
	return words

file_mapping_count = -1
prepro_data_dic = {}
count_to_name = {}
name_to_count = {}
file_titles = {}
toremove = []
ground_labels = []

with open('prepro_files.pkl') as f:
	prepro_data_dic = pickle.load(f)

with open('name_to_count.pkl') as f:
	name_to_count = pickle.load(f)

with open('count_to_name.pkl') as f:
	count_to_name = pickle.load(f)

with open('ground_labels.pkl') as f:
	ground_labels = pickle.load(f)

# ground_count = -1
# for i in os.listdir('20_newsgroups/'):
#     ground_count+=1
#     for j in sorted(os.listdir('20_newsgroups/'+i)):
#         file_mapping_count+=1
# #         print file_mapping_count
#         file_name = i+'/'+j
# #         print file_name
#         count_to_name[file_mapping_count] = file_name
#         name_to_count[file_name] = file_mapping_count
#         file_name_path = '20_newsgroups/'+i+'/'+j
#         temp_data = open(file_name_path,'rb').read().decode('utf-8', 'ignore').lower()
#         prepro_data = preprocess_input_sentence(temp_data)
#         prepro_data_dic[file_mapping_count] = prepro_data
#         ground_labels.append(ground_count)


In [3]:
# with open('prepro_files.pkl','wb') as f:
# 	f.write(pickle.dumps(prepro_data_dic)) 

# with open('name_to_count.pkl','wb') as f:
# 	f.write(pickle.dumps(name_to_count))

# with open('count_to_name.pkl','wb') as f:
# 	f.write(pickle.dumps(count_to_name))

# with open('ground_labels.pkl','wb') as f:
# 	f.write(pickle.dumps(ground_labels))

In [4]:
vocab = set()

for k,v in prepro_data_dic.iteritems():
    for i in range(len(v)):
        vocab.add(v[i])

In [5]:
len(vocab)

68215

In [6]:
# # Frequency of words in each document

# doc_word_freq_dic = {}
# for k,v in prepro_data_dic.iteritems():
#     doc_word_freq_dic[k] = {}
#     for i in range(len(v)):
#         if v[i] not in doc_word_freq_dic[k]:
#             doc_word_freq_dic[k][v[i]] = 0
#         doc_word_freq_dic[k][v[i]] += 1

In [7]:
bow = {}
# for k,v in prepro_data_dic.iteritems():
#     bow[k] = []
#     for i in range(len(vocab)):
#         if vocab[i] in v:
#             bow[k].append(doc_word_freq_dic[k][vocab[i]])
#         else:
#             bow[k].append(0)
# with open('bow.pkl','wb') as f:
#     f.write(pickle.dumps(bow))

with open('bow.pkl') as f:
    bow = pickle.load(f)

## Data Splitting Function

In [8]:
import random

def data_splitting(data,labels,ratio,seed):
    
    classwise_data = {}
    for i in range(len(labels)):
        if labels[i] not in classwise_data:
            classwise_data[labels[i]] = []
        classwise_data[labels[i]].append(data[i])
    
    final_train_data = []
    final_test_data = []
    final_train_labels = []
    final_test_labels = []
    final_train_docids = []
    final_test_docids = []
    
    for k,v in classwise_data.iteritems():
        shuf_ind = []
        for i in range(len(v)):
            shuf_ind.append(i)
        random.Random(seed).shuffle(shuf_ind)
        
        lim = int(len(shuf_ind)*ratio)
        train_ind = shuf_ind[:lim]
        test_ind = shuf_ind[lim:]

        train_data = []
        train_labels = []
        test_data = []
        test_labels = []
        for i in range(len(train_ind)):
            train_data.append(classwise_data[k][train_ind[i]])
            train_labels.append(k)
        for i in range(len(test_ind)):
            test_data.append(classwise_data[k][test_ind[i]])
            test_labels.append(k)
        final_train_data+=train_data
        final_test_data+=test_data
        final_train_labels+=train_labels
        final_test_labels+=test_labels
        
    return final_train_data,final_train_labels,final_test_data,final_test_labels


In [13]:
# Cosine similarity

import numpy as np

def cos_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

ratios = [0.5,0.8,0.9]
for rat in range(len(ratios)):
    print("Ratio: " + str(ratios[rat]))
    train_data,train_labels,test_data,test_labels = data_splitting(bow,ground_labels,ratios[rat],6)
    
    pred_lables = {}
    for i in range(len(test_data)):
        print("Test data: " + str(i))
        test_vector = np.array(test_data[i])
        sim_res = {}
        for j in range(len(train_data)):
            train_vector = np.array(train_data[j])
            simval = cos_sim(test_vector,train_vector)
            sim_res[j] = simval

        sorted_sim_results = sorted(sim_res.items(), key=operator.itemgetter(1),reverse=True)

        kvals = [1,3,5]
        for j in range(len(kvals)):
#             print "kvalue: " + str(kvals[j])
            majcount = {}
            topres = sorted_sim_results[:kvals[j]]
            for jj in range(len(topres)):
                predval = train_labels[topres[jj][0]]
                if predval not in majcount:
                    majcount[predval] = 0
                majcount[predval]+=1
            sort_maj = sorted(majcount.items(), key=operator.itemgetter(1),reverse=True)
            fpred = sort_maj[:1][0]
            if i not in pred_lables:
                pred_lables[i] = []
            pred_lables[i].append(fpred)
    
    corr_count1 = 0
    corr_count3 = 0
    corr_count5 = 0
    predicted_class1 = []
    predicted_class3 = []
    predicted_class5 = []
    for k,v in pred_lables.iteritems():
        k1_label = pred_lables[k][0]
        k3_label = pred_lables[k][1]
        k5_label = pred_lables[k][2]
        predicted_class1.append(k1_label)
        predicted_class3.append(k3_label)
        predicted_class5.append(k5_label)
        
        grnd_lab = test_labels[k]
        if k1_label == grnd_lab:
            corr_count1+=1
        if k3_label == grnd_lab:
            corr_count3+=1
        if k5_label == grnd_lab:
            corr_count5+=1
            
    print("Accuracy for k=1: " + str(((corr_count1*1.0)/len(test_data))*100) + "%")
    
    mat = np.zeros((len(predicted_class1), len(predicted_class1)))
    flag=0
    for it in range(len(predicted_class1)):
        flag+=1
        mat[int(test_labels[it])][int(predicted_class1[it])] += 1

    df = pd.DataFrame(mat)
    print("True vs Predicted")
    flag+=1
    print(mat)
    print("")
        
    print("Accuracy for k=3: " + str(((corr_count3*1.0)/len(test_data))*100) + "%")
    
    mat = np.zeros((len(predicted_class1), len(predicted_class1)))
    flag=0
    for it in range(len(predicted_class3)):
        flag+=1
        mat[int(test_labels[it])][int(predicted_class3[it])] += 1

    df = pd.DataFrame(mat)
    print("True vs Predicted")
    flag+=1
    print(mat)
    print("")
    
    print("Accuracy for k=5: " + str(((corr_count5*1.0)/len(test_data))*100) + "%")
    
    mat = np.zeros((len(predicted_class1), len(predicted_class1)))
    flag=0
    for it in range(len(predicted_class5)):
        flag+=1
        mat[int(test_labels[it])][int(predicted_class5[it])] += 1

    df = pd.DataFrame(mat)
    print("True vs Predicted")
    flag+=1
    print(mat)
    print("")
    
    with open('predicted1'+str(ratios[rat])+'.pkl','wb') as f:
        f.write(pickle.dumps(predicted_class1))
    
    with open('predicted3'+str(ratios[rat])+'.pkl','wb') as f:
        f.write(pickle.dumps(predicted_class3))
    
    with open('predicted5'+str(ratios[rat])+'.pkl','wb') as f:
        f.write(pickle.dumps(predicted_class5))



    

# cosine_sim_results = {}

# query_vector = np.array(query_vector)

# for k,v in tfresults.iteritems():
#     doc_vector = np.array(v)
#     simval = cos_sim(query_vector,doc_vector)
#     cosine_sim_results[k] = simval

Ratio: 0.5
Test data: 0
Test data: 1
Test data: 2
Test data: 3
Test data: 4
Test data: 5
Test data: 6
Test data: 7
Test data: 8
Test data: 9
Test data: 10
Test data: 11
Test data: 12
Test data: 13
Test data: 14
Test data: 15
Test data: 16
Test data: 17
Test data: 18
Test data: 19
Test data: 20
Test data: 21
Test data: 22
Test data: 23
Test data: 24
Test data: 25
Test data: 26
Test data: 27
Test data: 28
Test data: 29
Test data: 30
Test data: 31
Test data: 32
Test data: 33
Test data: 34
Test data: 35
Test data: 36
Test data: 37
Test data: 38
Test data: 39
Test data: 40
Test data: 41
Test data: 42
Test data: 43
Test data: 44
Test data: 45
Test data: 46
Test data: 47
Test data: 48
Test data: 49
Test data: 50
Test data: 51
Test data: 52
Test data: 53
Test data: 54
Test data: 55
Test data: 56
Test data: 57
Test data: 58
Test data: 59
Test data: 60
Test data: 61
Test data: 62
Test data: 63
Test data: 64
Test data: 65
Test data: 66
Test data: 67
Test data: 68
Test data: 69
Test data: 70
Test 

Test data: 553
Test data: 554
Test data: 555
Test data: 556
Test data: 557
Test data: 558
Test data: 559
Test data: 560
Test data: 561
Test data: 562
Test data: 563
Test data: 564
Test data: 565
Test data: 566
Test data: 567
Test data: 568
Test data: 569
Test data: 570
Test data: 571
Test data: 572
Test data: 573
Test data: 574
Test data: 575
Test data: 576
Test data: 577
Test data: 578
Test data: 579
Test data: 580
Test data: 581
Test data: 582
Test data: 583
Test data: 584
Test data: 585
Test data: 586
Test data: 587
Test data: 588
Test data: 589
Test data: 590
Test data: 591
Test data: 592
Test data: 593
Test data: 594
Test data: 595
Test data: 596
Test data: 597
Test data: 598
Test data: 599
Test data: 600
Test data: 601
Test data: 602
Test data: 603
Test data: 604
Test data: 605
Test data: 606
Test data: 607
Test data: 608
Test data: 609
Test data: 610
Test data: 611
Test data: 612
Test data: 613
Test data: 614
Test data: 615
Test data: 616
Test data: 617
Test data: 618
Test data:

Test data: 1093
Test data: 1094
Test data: 1095
Test data: 1096
Test data: 1097
Test data: 1098
Test data: 1099
Test data: 1100
Test data: 1101
Test data: 1102
Test data: 1103
Test data: 1104
Test data: 1105
Test data: 1106
Test data: 1107
Test data: 1108
Test data: 1109
Test data: 1110
Test data: 1111
Test data: 1112
Test data: 1113
Test data: 1114
Test data: 1115
Test data: 1116
Test data: 1117
Test data: 1118
Test data: 1119
Test data: 1120
Test data: 1121
Test data: 1122
Test data: 1123
Test data: 1124
Test data: 1125
Test data: 1126
Test data: 1127
Test data: 1128
Test data: 1129
Test data: 1130
Test data: 1131
Test data: 1132
Test data: 1133
Test data: 1134
Test data: 1135
Test data: 1136
Test data: 1137
Test data: 1138
Test data: 1139
Test data: 1140
Test data: 1141
Test data: 1142
Test data: 1143
Test data: 1144
Test data: 1145
Test data: 1146
Test data: 1147
Test data: 1148
Test data: 1149
Test data: 1150
Test data: 1151
Test data: 1152
Test data: 1153
Test data: 1154
Test dat

Test data: 1606
Test data: 1607
Test data: 1608
Test data: 1609
Test data: 1610
Test data: 1611
Test data: 1612
Test data: 1613
Test data: 1614
Test data: 1615
Test data: 1616
Test data: 1617
Test data: 1618
Test data: 1619
Test data: 1620
Test data: 1621
Test data: 1622
Test data: 1623
Test data: 1624
Test data: 1625
Test data: 1626
Test data: 1627
Test data: 1628
Test data: 1629
Test data: 1630
Test data: 1631
Test data: 1632
Test data: 1633
Test data: 1634
Test data: 1635
Test data: 1636
Test data: 1637
Test data: 1638
Test data: 1639
Test data: 1640
Test data: 1641
Test data: 1642
Test data: 1643
Test data: 1644
Test data: 1645
Test data: 1646
Test data: 1647
Test data: 1648
Test data: 1649
Test data: 1650
Test data: 1651
Test data: 1652
Test data: 1653
Test data: 1654
Test data: 1655
Test data: 1656
Test data: 1657
Test data: 1658
Test data: 1659
Test data: 1660
Test data: 1661
Test data: 1662
Test data: 1663
Test data: 1664
Test data: 1665
Test data: 1666
Test data: 1667
Test dat

Test data: 2119
Test data: 2120
Test data: 2121
Test data: 2122
Test data: 2123
Test data: 2124
Test data: 2125
Test data: 2126
Test data: 2127
Test data: 2128
Test data: 2129
Test data: 2130
Test data: 2131
Test data: 2132
Test data: 2133
Test data: 2134
Test data: 2135
Test data: 2136
Test data: 2137
Test data: 2138
Test data: 2139
Test data: 2140
Test data: 2141
Test data: 2142
Test data: 2143
Test data: 2144
Test data: 2145
Test data: 2146
Test data: 2147
Test data: 2148
Test data: 2149
Test data: 2150
Test data: 2151
Test data: 2152
Test data: 2153
Test data: 2154
Test data: 2155
Test data: 2156
Test data: 2157
Test data: 2158
Test data: 2159
Test data: 2160
Test data: 2161
Test data: 2162
Test data: 2163
Test data: 2164
Test data: 2165
Test data: 2166
Test data: 2167
Test data: 2168
Test data: 2169
Test data: 2170
Test data: 2171
Test data: 2172
Test data: 2173
Test data: 2174
Test data: 2175
Test data: 2176
Test data: 2177
Test data: 2178
Test data: 2179
Test data: 2180
Test dat

TypeError: int() argument must be a string or a number, not 'tuple'

In [70]:
sorted_sim_results[:1]

[(475, 0.9955619417500845)]

In [12]:
len(test_data)

2500

In [16]:
pred_lables[0]

[(0, 1), (0, 3), (0, 5)]

In [1]:
import random

In [25]:
import pickle as pkl
from collections import defaultdict

In [26]:
train_vectors,test_vectors = [],[]
with open('train_vectors.pkl') as f:
    train_vectors = pkl.load(f)
with open('train_vectors.pkl') as f:
    test_vectors = pkl.load(f)

In [27]:
for i in range(len(train_vectors)):
    list_data_points.append(train_vectors[i])
for i in range(len(test_vectors)):
    list_data_points.append(test_vectors[i])

In [28]:
length_list = len(list_data_points)
k=5
rand = random.sample(range(0,length_list),k)
centroids = []
for i in rand:
    centroids.append(list_data_points[i])

In [30]:
for k in range(10):
    groups = defaultdict(list)
    for i in list_data_points:
        min_d = 100000000
        grp_number = 0
        for number,j in enumerate(centroids):
            temp = dist(i,j)
            if temp < min_d:
                min_d = temp
                grp_number = j
        groups[grp_number].append(i)
    for grp in groups:
        centroid[grp] = np.mean(groups[grp])  
            

NameError: name 'dist' is not defined

In [31]:
def dist(x,y):
    t_num = np.dot(x, y)
    t_deno = np.linalg.norm(x)*np.linalg.norm(y)
    sim_scores = ((t_num)/((t_deno)*1.0))
    return sim_scores